<h1>Memory Usage Tools</h1>
<h2>By Thomas Hilton Johnson III and Brian Vanderwende</h2>

<p>The Notebook displayed here is mainly for the purpose of providing an available reference for NCAR HPC resource users to understand their memory usage over time. There is also code here written in bash for the purpose of of providing reference as to how to use tools to evaluate memory usage among other satistics when running jobs. This is to give users a better understanding of the tools available to help said users be as efficient and efective as possible when running jobs.</p>

<h3 style="background-color:cyan;text-align:center">Set Up</h3>
<p>Make sure that you have the appropriate modules loaded for this tutorial. The next set of commands will ensure such. For future reference please refer to CISL documentation on procedures and prerequisites for utilizing the tools found in this notebook.</p>

In [ ]:
module purge
ml intel openmpi peak_memusage arm-reports

<p>Next, check to make sure that the correct modules have been loaded.</p>

In [ ]:
module list

<p>Next, verify that there is file called <code>constructMultiDimensionalItem.f90</code> present in current directory. It is a prewritten Fortran script necessary for the rest of the operations in succeeding cells of the jupyter notebook to function. </p>

In [ ]:
ls

<p>If the <code>constructMultiDimensionalItem.f90</code> file is present, feel free to proceed to the next section.</p>

<h3 style="background-color:cyan;text-align:center">Submitting Jobs to Cheyenne or Casper</h3>
<p>Two job submission scripts have prepared as examples. Note that the job scripts are not interchangeable between the Casper and Cheyenne HPC systems. To submit a job on Casper use the comand <code>qsub</code> followed by your prepared script for running the job as an argument. For submitting jobs on Cheyenne, use <code>sbatch</code> followed with the necessary scipt for the job as an argument.</p>

<p><code>cat cheyenne_fortran_script_for_execution.sh</code> is the prepared job script for Cheyenne. <code>casper_fortran_script_for_execution.sh</code> is the prepared job script for Casper. Note that you must modify the account field with the account that you are allowed to access core hours with for the scripts to be usable.</p>

<p>This an example of of a Cheyenne job submission script using the Fortran script that was prepared for this tutorial:</p>

In [2]:
cat cheyenne_fortran_script_for_execution.sh

#!/bin/tcsh
## Authors: Thomas Johnson III and Brian Vanderwende
## Script that displays output of Peak Memory Usage for the Fortran Script on Cheyenne
#PBS -N fortran_cheyenne_peakmemusage
#PBS -A <account>
#PBS -l walltime=00:17:00
#PBS -q regular
#PBS -j oe
#PBS -m abe
#PBS -l select=1:ncpus=1

setenv TMPDIR /glade/scratch/$USER/temp
mkdir -p $TMPDIR

ifort -o execution_code_fortran constructMultiDimensionalItem.f90
./execution_code_fortran


<p>This an example of of a Casper job submission script using the Fortran script that was prepared for this tutorial:</p>

In [3]:
cat casper_fortran_script_for_execution.sh

#!/bin/bash -l
## Authors: Thomas Johnson III and Brian Vanderwende
#SBATCH --job-name=fortrancasperjobsubmission
#SBATCH --<account>
#SBATCH --ntasks=1
#SBATCH --ntasks-per-node=1
#SBATCH --time=00:17:00
#SBATCH --partition=dav
#SBATCH -o fortran_casper_job_output.%j

export TMPDIR=/glade/scratch/$USER/temp
mkdir -p $TMPDIR

ifort -o execution_code_fortran constructMultiDimensionalItem.f90
export OMP_NUM_THREADS=1
srun ./execution_code_fortran_pile 80 80 80


<p>An example of how to run a job on Cheyenne using the <code>qsub</code> command.</p>

In [ ]:
qsub cheyenne_fortran_script_for_execution.sh

<p>The output from the previous cell should contain the job ID, which can be useful for specific information concerning the next part of the notebook. For Cheyenne's job statistics, you would utilize <code>qstat -xf</code> with the <code>grep</code> command to pull out specific data. Note that Cheyenne and Casper are not identical systems. The <code>grep</code> command will be used for the purpose fo filtering out desired information from the collection of output loaded from the qstat command. Using the combination of <code>qstat -xf</code> and <code>grep</code> should yield desired results. <code>qstat -j</code> allows for statistics for a specific job to be pulled up.</p>

In [ ]:
qstat -xf [jobid] 

In [ ]:
sbatch casper_fortran_script_for_execution.sh

<p>There will be output that includes a job id. You can in turn track the status of the job with <code>squeue -j [jobid]</code>. After the job finishes, you can run <code>sacct -j [jobid] --format=[Arguments] --units=GB</code> to find statistics about the job. Various arguments can be used to display information of the job from the time that it took the job to run to the resources that were utilized to run that job. An example of sacct is: <code>sacct -j [jobid] --format=jobid,elapsed,ncpus,ReqMem,CPUTime,MaxRSS --units=GB</code>.</p>
<p>To view the job statistics for the job submitted to Casper, substitute in the job id for the code that is below and run the code.</p>

In [ ]:
sacct -j [jobid] --format=jobid,elapsed,ncpus,ReqMem,CPUTime,MaxRSS --units=GB

<p>The output currently shows the jobid, the time elapsed for the job, the number of CPUs used for the job, The Requested Memory for the job, The CPUTime that was taken to run the job, and maximum amount of memory that the job required. There are other pieces of information that can be displayed by adding the correct keywords to <code>--format</code>.</p>

<p>Note that the <code>--units</code> argument is used to determine the units to be displayed, such as <code>GB</code> for Gigabytes <code>MB</code>Megabytes.</p>


<h3 style="background-color:cyan;text-align:center">Starting Within an Interactive Job</h3>
<p>The before mentioned tools to be taught that need a job to be ran in order to be used. One way to run jobs is to send scripts with all the necessary details to be queued and ran by the machine. Another way to do such is to run an interactive job, in which you are hands-on in constructing and defining the job.</p>

<p>To start using the fortran code, first we must compile it and generate an executable. If using the Intel compiler, use ifort. If using gfortran, use gfortran. The next part of the tutorial demonstrates using ifort with flags and arguments to compile the fortran code.</p>

In [1]:
ifort -g -o execution_code_fortran_pile constructMultiDimensionalItem.f90 

bash: ifort: command not found


: 127

<p>An executable should have been generated from compilation called <code>execution_code_fortran_pile</code>. The <code>-g</code> flag activates debugging within the compiler to point out any errors or bugs in the Fortran code during compilation. The <code>-o</code> flag is used for compiling. After the flag options the executable for the Fortran code is specified first followed by the Fortran files that will be utilized to build the executable. Now, for the next part. We will run the executable with the correct arguments. The first tool we will use is Peak Memory Usage. To Use this tool it is necessary to first load the Peak Memory Usage Module using <code>module load intel impi peak_memusage</code>. When running the Fortran executable, we must also add the <code>peak_memusage.exe</code> executable before the Fortran executable. There are also four integer arguments that must be placed, the first three to determine the size of each array in the multidimensional array structure and the last integer arguemnt to determine the number of iterations of the code. The arguments are added after the Fortran executable. The following section of code is the correct means to set up said code execution.</p>

In [ ]:
peak_memusage.exe ./execution_code_fortran_pile 80 80 80 100

<p>The previous output is the result of running the executable compiled from the Fortran code. Do note that the time it takes to process said output is dependent on the three integer inputs that are give. The larger the integer inputs, the more time that the processing will take in comparison to smaller integer inputs. Also, the larger the integer inputs, the more memory that will be required to run the executable. After the program's output is displayed, the output of the Peak Memory Usage tool will be displayed showing the memory that was required to run the program.</p>

<p>The next tool to be displayed is an Arm Performance Report that is made possible by loading the <code>arm-reports</code> module. This module enables users to generate an HTML and text file, displaying identical information, as to various statisics regarding the executable that has been ran.</p>

<p>Next we will generate a performance report. To do so, add perf-report before the executable and the executable's arguments. This will ensure that the job is ran and will output HTML and text files to store information regarding the perfromance of the code that has been ran.</p>

In [ ]:
perf-report ./execution_code_fortran_pile 80 80 80 100

<p>The output of the perfromance report will not be displayed merely as output. Instead, two files will be created within the current directory. One of the files is a text file and the other file is an HTML file. The files contain similar content which includes statistics concerning the memory, energy, CPU usage, and Input/Output data. The purpose fo this data is to provide the user with information that is relevant to observing the charactersistics of the code when ran.</p>
<p>The performance reports should be displayed in the current directory. To find the performance report, use <code>ls</code> and locate the HTML or text file that is now present (example: execution_code_fortran_pile_1p_1n_1t_2019-07-19_11-05.txt). After finding those files, please cat the text file (for simplicity of appearance) by replacing the file argument in the cell below the <code>ls</code> command cell with the name of the text file holding the results of the performance report.</p>

In [ ]:
ls

In [ ]:
cat [performance_report_file_argument.txt]

<p>For more information, please refer to the module help information, reference documentation on the software module being utilized, or contact the CISL Consulting Group.</p>